<a href="https://colab.research.google.com/github/ucfilho/Raianars_paper_one_revisited/blob/main/After_asked_reviews_one_2022/LSHADE/LSHADEv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import scipy.stats
import random

In [2]:
cols=30
dim = cols
NPAR1 = 100
ITERATIONS = 1000
MAX=np.repeat(100,dim)
MIN=np.repeat(-100,dim)
popsize = NPAR1
H = 6
kH = 0 # kH is used to work with H  
terminal = 0.1
p=0.1
SF=[];SCR=[];MF=[];MCR=[];Xarq=[]

In [3]:
# HappyCat function
# global mimima 0 at (-1,-1,...,-1)
# Not convex and non separable it is usually evaluated at xi =[(-600,600),..(-600,600)]

def fobj(x): 
    alpha=1. / 8
    s = sum(x**2) 
    return ((s - len(x))**2)**alpha + (s / 2 + sum(x)) / len(x) + 0.5 

In [4]:
def LShade(MAX,MIN, popsize,fobj,setTUNE,best,fbest,fitness,X,Xarq,FES):

  SF,SCR,MF,MCR,p, terminal, Narquive,H,kH = setTUNE

  fmutant = []
  Xmutant = []
  fx=[]; fu=[] # to select the wij Lehmer Mean.
  # setTUNE = [SF,SCR,MF,MCR,p, terminal, Narquive,H] 
  dim = X.shape[1]
  #best_number = int(p*popsize)
  if (p*popsize<1):
    best_number=1
  else:
    best_number = int(p*popsize)
  justGoOUT=1

  if(justGoOUT==1):
    SCR =[];SF=[];
    for i in range(popsize):
      ri = random.randint(0,H-1) # line 7
      miF = MF[ri]
      miCR = MCR[ri]
      if(MCR[ri] == terminal):
        miCR = 0
        crossp = 0
      else:
        crossp = np.random.normal(miCR , 0.1)
      while (True):
        mut = scipy.stats.cauchy.rvs(loc=miF, scale=0.1)
        if (mut > 0):
          break
      if(mut > 1.0): mut =1.0
      if(crossp > 1.0): crossp =1
      if(crossp < 0): crossp = 0 

      ind = fitness.argsort()[range(best_number)] # find index of best p*popsize
      best_idx = random.choice(ind) # index of the best in p*popsize (random best)
      pbest = X[best_idx] # random best 
      idxs = [idx for idx in range(popsize) if idx != i]
      iused = np.random.choice(idxs, 1, replace = False)
      idxs = [idx for idx in range(popsize) if (idx != iused and idx != best_idx)]
      a  = X[iused]
      if(len(Xarq)>0):
        Xii = np.asarray(Xarq)
        Xnew = np.append(X[idxs,:],Xii,axis=0)
      else:
        Xnew=np.copy(X[idxs,:])

      idxs2 = [idx for idx in range(len(Xnew[:,0])) if 2 > 1]
      b  = Xnew[np.random.choice(idxs2, 1, replace = False)]

      mutant = X[i,:]+mut*(pbest-X[i,:]) + mut * (b - a)
      mutant = mutant.ravel()

      #trial = np.copy(mutant)
      trial = np.copy(X[i,:])

      jrand = random.randint(0,dim-1)

      for j in range(dim):
        rdn = np.random.rand()
        if(rdn < crossp or j == jrand) : 
          trial[j] = mutant[j] 
        else:
          trial[j] = X[i,j]

      mutant  = np.copy(trial) # just to use word mutant instead of trial
      
      for k in range(dim):
        if(mutant[k]>MAX[k]):
          mutant[k]=MAX[k]
        if(mutant[k]<MIN[k]):
          mutant[k]=MIN[k]

      Xmutant.append(mutant)

      FES = FES + 1
      fmut =fobj(mutant)
      fmutant.append(fmut)
    
    

      #for i in range(popsize):
      if( fmutant[i]  <= fitness[i]):
        Xold = X[i,:] 
        X[i,:] = Xmutant[i]
        fx.append(fitness[i])
        fu.append(fmutant[i])
        fitness[i]=fmutant[i]
        SCR.append(crossp)
        SF.append(mut)

        sizeARQ = np.array(Xarq).shape[0]
        if(sizeARQ < Narquive):
          Xarq.append(Xold)
        else:
          ri = random.randint(0, Narquive-1) # deleting individuals if necessary line 19
          Xarq[ri] = Xold

        if(fbest >= fmutant[i]):
          best = Xmutant[i]
          fbest = fmutant[i]
      else:
        X[i,:] = X[i,:] # it is not necessary this line but just to remeber it

    
    # memory update algorithm 1:
    if(kH > H-1): kH = 0

    if (len(SCR)>0):
      deltaF = np.abs(np.asarray(fu)-np.asarray(fx))
      if( sum(deltaF) > 0):
        w = deltaF/sum(deltaF)
      else:
        w =np.ones(len(fu))
      if(MCR[kH] == terminal or max(SCR)==0):
        MCR[kH] = terminal
      else:
        y = np.asarray(SCR)
        MCR[kH] = np.dot(w,y**2)/np.dot(w,y)
      y = np.asarray(SF)
      MF[kH] = np.dot(w,y**2)/np.dot(w,y)
      kH = kH + 1

    else:
      MCR = MCR
      MF = MF

  setTUNE =[SF,SCR,MF,MCR,p, terminal, Narquive,H,kH]


  y=fitness


  XY= np.c_[X,y] #concatena x e y em 2 colunas            
  XYsorted = XY[XY[:,-1].argsort()] #Ordena a partir da last col(Y) for all row
  X=XYsorted[:,0:dim]
  XY=XYsorted
  fitness = XYsorted[:,-1]  
  BEST=X[0,:]
  FOBEST=fitness[0]
  BEST_XY =np.append(BEST,FOBEST)
  
  return fitness,X,BEST,FOBEST,XY,BEST_XY,FES,Xarq,setTUNE


In [5]:

X=np.zeros((NPAR1,cols)) # X1 is Frannk population
for i in range(NPAR1):
  for j in range(cols):
    r=np.random.random()
    X[i,j]=r*(MAX[j]-MIN[j])+MIN[j]

FES = 0
X1 = np.copy(X)
fit1 = np.asarray([fobj(ind) for ind in X1])
FES = FES + NPAR1
best_idx = np.argmin(fit1)
best = X1[best_idx]
fbest = fit1[best_idx] 
BEST1,FOBEST1 = best, fbest

In [6]:
for i in range(H):
  MCR.append(0.5);MF.append(0.5) # line 3
Narquive = popsize
setTUNE =[SF,SCR,MF,MCR,p, terminal, Narquive,H,kH]

In [7]:
for i in range(ITERATIONS):
  
  fit1,X1,BEST1,FOBEST1,XY,BEST_XY,FES,Xarq,setTUNE= LShade(MAX,MIN, popsize,fobj,setTUNE,best,fbest,fit1,X1,Xarq,FES)
  print('i=',i,FOBEST1)


i= 0 670.829922025249
MCR= [0.5070729359217766, 0.5, 0.5, 0.5, 0.5, 0.5]
MF= [0.5028818282653037, 0.5, 0.5, 0.5, 0.5, 0.5]
i= 100 0.30893146523882764
MCR= [0.8774221086807219, 0.9909947512143562, 0.9144850066273595, 0.9692878864820171, 0.8982163463944294, 0.9361462920746001]
MF= [0.5723521061155652, 0.3255518164889028, 0.4409299473731206, 0.009716726764781192, 0.45026616032505956, 0.4513643872631984]
i= 200 0.2811515473945914
MCR= [0.6255609908451987, 0.5232328632305697, 0.6510430461828551, 0.5778185046727665, 0.585221074027816, 0.9429368157154722]
MF= [0.0139506557164073, 0.3206509079480997, 0.2465864363869045, 0.1553080151650399, 0.2517533129790736, 0.5580946410032372]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:120: RuntimeWarning: invalid value encountered in double_scalars


i= 300 0.23731356774942625
MCR= [0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
MF= [0.07172241031252481, 0.5311387770106426, 0.06210032057205331, 0.18651200248639213, 0.124556711800192, 0.3266194660493543]
i= 400 0.19960889973940943
MCR= [0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
MF= [0.010549151050877965, 0.04633987364493812, 0.028899500932382423, 0.04050750158954097, 0.008953805791798306, 0.1064677136876957]
i= 500 0.19960889973940943
MCR= [0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
MF= [0.094788699574611, 0.025766793503816384, 0.35147665717523563, 0.03167404297247531, 0.015640646135558163, 0.12092982728646182]
i= 600 0.19960889973940943
MCR= [0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
MF= [0.22086241906763104, 0.10482031669669234, 0.007962972840945983, 0.1344576624388071, 0.0019707345172850013, 0.04394004981691818]
i= 700 0.19960889973940943
MCR= [0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
MF= [0.03425576863023647, 0.00032888818938489833, 0.008411270211101628, 0.00015306902181491948, 0.0004378478614697461, 0.013077054036516163]
i= 800 0.17243976823

In [8]:
X1.shape

(100, 30)

In [9]:
fobj(X1[0,:])

0.14495655295291276

In [10]:
fit1

array([0.14495655, 0.1708399 , 0.17447825, 0.18158236, 0.19342162,
       0.20111377, 0.20116127, 0.20213604, 0.22553408, 0.22827211,
       0.2301372 , 0.23101423, 0.23396434, 0.23563859, 0.2361682 ,
       0.23626993, 0.23731357, 0.23887641, 0.24053046, 0.24405988,
       0.24661137, 0.24680428, 0.24700838, 0.24789331, 0.24888127,
       0.25389504, 0.25411912, 0.25576265, 0.25705895, 0.25995418,
       0.26039076, 0.26111959, 0.26189377, 0.26201348, 0.26298118,
       0.26727378, 0.26874353, 0.26993094, 0.27107393, 0.27178527,
       0.27537236, 0.27641245, 0.27964986, 0.281671  , 0.28282956,
       0.2838834 , 0.28397505, 0.28521594, 0.28580778, 0.28710511,
       0.28733676, 0.28768574, 0.29304851, 0.30012473, 0.30062573,
       0.30103581, 0.30233732, 0.30348626, 0.30420698, 0.30630658,
       0.30714757, 0.31054963, 0.31117641, 0.31319671, 0.31353865,
       0.31823716, 0.31909978, 0.32072527, 0.32184361, 0.32737485,
       0.32771015, 0.32852409, 0.32919057, 0.33448772, 0.33547

In [11]:
setTUNE

[[],
 [],
 [0.0227011144029972,
  0.006060499394480394,
  0.0013890635118180442,
  0.004189088437630407,
  0.01944824619688883,
  0.007175227071779155],
 [0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
 0.1,
 0.1,
 100,
 6,
 1]

In [12]:
setTUNE[2]# =[SF,SCR,MF,MCR,p, terminal, Narquive,H,kH]

[0.0227011144029972,
 0.006060499394480394,
 0.0013890635118180442,
 0.004189088437630407,
 0.01944824619688883,
 0.007175227071779155]

In [13]:
setTUNE[3]# =[SF,SCR,MF,MCR,p, terminal, Narquive,H,kH]

[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]